In [15]:
import logging
import math
from dataclasses import dataclass
from statistics import mean
from typing import Dict, List, Optional, Tuple

In [16]:
RATIO_FAMILY = {
    "debt_ebitda": "leverage",
    "net_debt_ebitda": "leverage",
    "debt_equity": "leverage",
    "debt_capital": "leverage",
    "ffo_debt": "leverage_rev",
    "fcf_debt": "leverage_rev",
    "interest_coverage": "coverage",
    "fixed_charge_coverage": "coverage",
    "dscr": "coverage",
    "ebitda_margin": "profit",
    "ebit_margin": "profit",
    "roa": "profit",
    "roe": "profit",
    "capex_dep": "other",
    "current_ratio": "other",
    "rollover_coverage": "other",
    "altman_z": "altman",
}

In [17]:
SCORE_TO_RATING: List[Tuple[float, str]] = [
    (95, "AAA"),
    (90, "AA+"),
    (85, "AA"),
    (80, "AA-"),
    (75, "A+"),
    (70, "A"),
    (65, "A-"),
    (60, "BBB+"),
    (55, "BBB"),
    (50, "BBB-"),
    (45, "BB+"),
    (40, "BB"),
    (35, "BB-"),
    (30, "B+"),
    (25, "B"),
    (20, "B-"),
    (15, "CCC+"),
    (10, "CCC"),
    (5,  "CCC-"),
    (2,  "CC"),
    (0,  "C"),
]


In [18]:
RATING_SCALE = [
    "AAA", "AA+", "AA", "AA-",
    "A+", "A", "A-",
    "BBB+", "BBB", "BBB-",
    "BB+", "BB", "BB-",
    "B+", "B", "B-",
    "CCC+", "CCC", "CCC-",
    "CC", "C",
]


In [19]:
RATING_WEIGHTS = {
    "quantitative": None,   # if None, auto-weight by count
    "qualitative": None,
}

In [20]:
DISTRESS_TRIGGERS = {
    "interest_coverage": 1.0,
    "dscr": 1.0,
    "altman_z": 1.81,
}

DISTRESS_BANDS = {
    "interest_coverage": [
        (0.5, -4),
        (0.8, -3),
        (1.0, -2),
    ],
    "dscr": [
        (0.8, -3),
        (0.9, -2),
        (1.0, -1),
    ],
    "altman_z": [
        (1.2, -4),
        (1.5, -3),
        (1.81, -2),
    ],
}

MAX_DISTRESS_NOTCHES = -4  # floor for cumulative distress notches

In [21]:
# Ratio grids: (low, high, score)
RATIO_GRIDS: Dict[str, List[Tuple[float, float, float]]] = {
    "debt_ebitda": [
        (float("-inf"), 2.0, 100),
        (2.0, 3.0, 75),
        (3.0, 4.0, 50),
        (4.0, 6.0, 25),
        (6.0, float("inf"), 0),
    ],
    "net_debt_ebitda": [
        (float("-inf"), 1.5, 100),
        (1.5, 3.0, 75),
        (3.0, 4.5, 50),
        (4.5, 6.0, 25),
        (6.0, float("inf"), 0),
    ],
    "ffo_debt": [
        (0.40, float("inf"), 100),
        (0.25, 0.40, 75),
        (0.12, 0.25, 50),
        (0.0, 0.12, 25),
        (float("-inf"), 0.0, 0),
    ],
    "fcf_debt": [
        (0.20, float("inf"), 100),
        (0.10, 0.20, 75),
        (0.0, 0.10, 50),
        (-0.10, 0.0, 25),
        (float("-inf"), -0.10, 0),
    ],
    "debt_equity": [
        (float("-inf"), 0.5, 100),
        (0.5, 1.0, 75),
        (1.0, 2.0, 50),
        (2.0, 4.0, 25),
        (4.0, float("inf"), 0),
    ],
    "debt_capital": [
        (float("-inf"), 0.20, 100),
        (0.20, 0.35, 75),
        (0.35, 0.50, 50),
        (0.50, 0.70, 25),
        (0.70, float("inf"), 0),
    ],
    "interest_coverage": [
        (8.0, float("inf"), 100),
        (5.0, 8.0, 75),
        (3.0, 5.0, 50),
        (1.5, 3.0, 25),
        (float("-inf"), 1.5, 0),
    ],
    "fixed_charge_coverage": [
        (6.0, float("inf"), 100),
        (4.0, 6.0, 75),
        (2.5, 4.0, 50),
        (1.5, 2.5, 25),
        (float("-inf"), 1.5, 0),
    ],
    "dscr": [
        (2.0, float("inf"), 100),
        (1.5, 2.0, 75),
        (1.2, 1.5, 50),
        (1.0, 1.2, 25),
        (float("-inf"), 1.0, 0),
    ],
    "ebitda_margin": [
        (0.25, float("inf"), 100),
        (0.15, 0.25, 75),
        (0.10, 0.15, 50),
        (0.05, 0.10, 25),
        (float("-inf"), 0.05, 0),
    ],
    "ebit_margin": [
        (0.15, float("inf"), 100),
        (0.10, 0.15, 75),
        (0.05, 0.10, 50),
        (0.0, 0.05, 25),
        (float("-inf"), 0.0, 0),
    ],
    "roa": [
        (0.12, float("inf"), 100),
        (0.08, 0.12, 75),
        (0.04, 0.08, 50),
        (0.0, 0.04, 25),
        (float("-inf"), 0.0, 0),
    ],
    "roe": [
        (0.20, float("inf"), 100),
        (0.12, 0.20, 75),
        (0.05, 0.12, 50),
        (0.0, 0.05, 25),
        (float("-inf"), 0.0, 0),
    ],
    "capex_dep": [
        (1.2, 1.8, 100),
        (0.9, 1.2, 75),
        (1.8, 2.5, 75),
        (0.7, 0.9, 50),
        (2.5, 3.5, 50),
        (0.5, 0.7, 25),
        (3.5, float("inf"), 25),
        (float("-inf"), 0.5, 0),
    ],
    "current_ratio": [
        (2.0, float("inf"), 100),
        (1.5, 2.0, 75),
        (1.0, 1.5, 50),
        (0.7, 1.0, 25),
        (float("-inf"), 0.7, 0),
    ],
    "rollover_coverage": [
        (2.0, float("inf"), 100),
        (1.2, 2.0, 75),
        (0.8, 1.2, 50),
        (0.5, 0.8, 25),
        (float("-inf"), 0.5, 0),
    ],
    "altman_z": [
        (3.0, float("inf"), 100),
        (2.7, 3.0, 75),
        (1.8, 2.7, 50),
        (1.5, 1.8, 25),
        (float("-inf"), 1.5, 0),
    ],
}

In [22]:
QUAL_SCORE_SCALE: Dict[int, float] = {
    5: 100.0,
    4: 75.0,
    3: 50.0,
    2: 25.0,
    1: 0.0,
}

In [23]:
# =========================
# Helper functions
# =========================

def score_ratio(name: str, value: float) -> Optional[float]:
    grid = RATIO_GRIDS.get(name)  # fetch grid for given ratio
    if not grid or value is None or math.isnan(value):
        return None
    for low, high, score in grid:
        if low <= value < high:
            return float(score)
    return None


In [24]:
def score_qual_factor_numeric(value: int) -> Optional[float]:
    return QUAL_SCORE_SCALE.get(int(value))  # map 1–5 to score


In [25]:
def compute_altman_z_from_components(
    working_capital: float,
    total_assets: float,
    retained_earnings: float,
    ebit: float,
    market_value_equity: float,
    total_liabilities: float,
    sales: float,
) -> float:
    if total_assets == 0 or total_liabilities == 0:
        return float("nan")
    A = working_capital / total_assets
    B = retained_earnings / total_assets
    C = ebit / total_assets
    D = market_value_equity / total_liabilities
    E = sales / total_assets
    return 1.2 * A + 1.4 * B + 3.3 * C + 0.6 * D + 1.0 * E

In [26]:
def compute_peer_score(
    fin_current: Dict[str, float],
    peers: Dict[str, List[float]],
) -> Optional[float]:
    under = 0   # count ratios underperforming peers
    total = 0   # count comparable ratios
    for rname, peer_vals in peers.items():
        if rname not in fin_current or not peer_vals:
            continue
        cp = fin_current[rname]
        peer_avg = mean(peer_vals)
        if peer_avg == 0:
            continue
        total += 1
        if cp < peer_avg * 0.9:
            under += 1
    if total == 0:
        return None
    under_share = under / total
    if under_share <= 0.10:
        return 100.0
    elif under_share <= 0.30:
        return 75.0
    elif under_share <= 0.60:
        return 50.0
    elif under_share <= 0.80:
        return 25.0
    else:
        return 0.0


In [27]:
def score_to_rating(score: float) -> str:
    for cutoff, grade in SCORE_TO_RATING:
        if score >= cutoff:      # first cutoff satisfied wins
            return grade
    raise ValueError(f"Score {score} did not match any cutoff")

In [28]:
def safe_score_to_rating(score: float) -> str:
    try:
        return score_to_rating(score)
    except ValueError as e:
        logging.error("Score-to-rating mapping failed: %s", e)
        return "N/R"

In [29]:
def move_notches(grade: str, notches: int) -> str:
    if grade not in RATING_SCALE:
        return grade
    idx = RATING_SCALE.index(grade)
    new_idx = max(0, min(idx - notches, len(RATING_SCALE) - 1))  # clamp to scale
    return RATING_SCALE[new_idx]


In [30]:
def apply_sovereign_cap(
    issuer_grade: str,
    sovereign_grade: Optional[str],
) -> str:
    if sovereign_grade is None:
        return issuer_grade
    if issuer_grade not in RATING_SCALE or sovereign_grade not in RATING_SCALE:
        return issuer_grade
    i = RATING_SCALE.index(issuer_grade)
    s = RATING_SCALE.index(sovereign_grade)
    return RATING_SCALE[max(i, s)]  # worse (higher index) of issuer vs sovereign

In [31]:
def compute_effective_weights(n_quant: int, n_qual: int) -> Tuple[float, float]:
    """
    Determine effective quantitative vs qualitative weights.

    Priority:
    1) If both weights are explicitly configured in RATING_WEIGHTS, use them.
    2) Otherwise, weight automatically in proportion to the number of active
       quantitative and qualitative items.
    3) If there are no active items at all, return (0.0, 0.0) and let the caller
       decide how to handle this degenerate case.
    """
    wq_cfg = RATING_WEIGHTS["quantitative"]
    wl_cfg = RATING_WEIGHTS["qualitative"]

    # 1) Hard-configured weights take precedence
    if wq_cfg is not None and wl_cfg is not None:
        return float(wq_cfg), float(wl_cfg)

    # 2) Automatic weighting based on factor counts
    n_quant = max(n_quant, 0)
    n_qual = max(n_qual, 0)
    total = n_quant + n_qual

    if total == 0:
        # Degenerate case: no usable quantitative or qualitative factors
        # Caller is expected to treat the resulting combined score as non-informative
        return 0.0, 0.0

    wq = n_quant / total
    wl = n_qual / total
    return wq, wl

In [32]:
def get_rating_band(rating: str) -> Tuple[float, float]:
    # returns inclusive score band [min, max] that maps to rating
    for i, (cutoff, grade) in enumerate(SCORE_TO_RATING):
        if grade == rating:
            band_min = cutoff
            if i == 0:
                band_max = 100.0
            else:
                prev_cutoff, _ = SCORE_TO_RATING[i - 1]
                band_max = prev_cutoff - 1.0
            return band_min, band_max
    raise ValueError(f"Unknown rating grade: {rating!r}")

In [33]:
def derive_outlook_band_only(combined_score: float, rating: str) -> str:
    """
    Band-based outlook on the base rating, using floored score.
    """
    band_min, band_max = get_rating_band(rating)
    cs = math.floor(combined_score)
    if cs == band_max:
        return "Positive"
    elif cs == band_min:
        return "Negative"
    else:
        return "Stable"

In [34]:
def derive_outlook_with_distress_trend(
    base_outlook: str,
    distress_notches: int,
    fin_t0: Dict[str, float],
    fin_t1: Dict[str, float],
) -> str:
    """
    Adjust outlook when hardstops are active based on trend in distress ratios.
    """
    if distress_notches >= 0:
        return base_outlook

    ratios = ["interest_coverage", "dscr", "altman_z"]
    improving = False
    deteriorating = False

    for r in ratios:
        v0 = fin_t0.get(r)
        v1 = fin_t1.get(r)
        if v0 is None or v1 is None:
            continue
        # higher is better
        if v0 > v1:
            improving = True
        elif v0 < v1:
            deteriorating = True

    if improving and not deteriorating:
        return "Stable"
    if deteriorating and not improving:
        return "Negative"
    return "Stable"

In [35]:
# =========================
# Data classes
# =========================

@dataclass
class QuantInputs:
    fin_t0: Dict[str, float]             # current period
    fin_t1: Dict[str, float]             # previous period
    fin_t2: Dict[str, float]             # two periods ago
    components_t0: Dict[str, float]
    components_t1: Dict[str, float]
    components_t2: Dict[str, float]
    peers_t0: Dict[str, List[float]]


In [36]:
@dataclass
class QualInputs:
    factors_t0: Dict[str, int]           # 1–5 values
    factors_t1: Dict[str, int]


In [37]:
@dataclass
class RatingOutputs:
    issuer_name: str
    quantitative_score: float
    qualitative_score: float
    combined_score: float
    peer_score: Optional[float]
    base_rating: str                     # model rating before hardstops / cap
    distress_notches: int
    hardstop_rating: str                 # rating after distress notches
    capped_rating: str                   # rating after sovereign cap
    final_rating: str                    # delivered rating (currently = capped)
    hardstop_triggered: bool
    hardstop_details: Dict[str, float]
    sovereign_rating: Optional[str]
    sovereign_outlook: Optional[str]
    sovereign_cap_binding: bool          # True if final_rating == sovereign_rating
    outlook: str
    bucket_avgs: Dict[str, float]
    altman_z_t0: float
    flags: Dict[str, bool]
    rating_explanation: str

In [38]:
# =========================
# Model
# =========================

class RatingModel:
    def __init__(self, cp_name: str):
        self.cp_name = cp_name

    def _ensure_altman_z(self, fin: Dict[str, float], comps: Dict[str, float]) -> float:
        # compute Altman Z if not already present
        if "altman_z" in fin and fin["altman_z"] is not None:
            return fin["altman_z"]
        z = compute_altman_z_from_components(
            comps["working_capital"],
            comps["total_assets"],
            comps["retained_earnings"],
            comps["ebit"],
            comps["market_value_equity"],
            comps["total_liabilities"],
            comps["sales"],
        )
        fin["altman_z"] = z
        logging.info("%s-AltmanZ: computed z=%.3f from components", self.cp_name, z)
        return z

    def compute_quantitative(
        self,
        q: QuantInputs,
    ) -> Tuple[float, Optional[float], Dict[str, float], float, int]:
        fin = dict(q.fin_t0)  # copy to avoid mutating caller
        altman_z = self._ensure_altman_z(fin, q.components_t0)

        scores: List[float] = []
        bucket_scores: Dict[str, List[float]] = {
            "leverage": [],
            "leverage_rev": [],
            "coverage": [],
            "profit": [],
            "other": [],
            "altman": [],
        }

        n_quant_items = 0

        for rname, val in fin.items():
            if rname not in RATIO_FAMILY:
                continue
            s = score_ratio(rname, val)
            if s is None:
                logging.info("%s-Quant: no grid/score for ratio %s", self.cp_name, rname)
                continue
            scores.append(s)
            n_quant_items += 1
            family = RATIO_FAMILY[rname]
            bucket_scores.setdefault(family, []).append(s)
            logging.info(
                "%s-Quant: %s value=%.3f score=%.1f family=%s",
                self.cp_name, rname, val, s, family
            )

        peer_score = compute_peer_score(fin, q.peers_t0)
        if peer_score is not None:
            scores.append(peer_score)
            n_quant_items += 1
            bucket_scores["other"].append(peer_score)
            logging.info("%s-PeerPositioning: score=%.1f", self.cp_name, peer_score)

        quantitative_score = sum(scores) / len(scores) if scores else 0.0
        logging.info("%s-Quant: aggregate score=%.1f", self.cp_name, quantitative_score)

        bucket_avgs = {
            b: round(sum(vals) / len(vals), 1) if vals else 0.0
            for b, vals in bucket_scores.items()
        }

        return quantitative_score, peer_score, bucket_avgs, altman_z, n_quant_items

    def compute_qualitative(self, ql: QualInputs) -> Tuple[float, int]:
        scores: List[float] = []
        n_qual_items = 0
        for name, val in ql.factors_t0.items():
            s = score_qual_factor_numeric(val)
            if s is None:
                logging.info(
                    "%s-Qual: unknown or out-of-range factor %s=%s",
                    self.cp_name, name, val
                )
                continue
            scores.append(s)
            n_qual_items += 1
            logging.info(
                "%s-Qual: %s=%s score=%.1f",
                self.cp_name, name, val, s
            )
        qualitative_score = sum(scores) / len(scores) if scores else 0.0
        logging.info("%s-Qual: aggregate score=%.1f", self.cp_name, qualitative_score)
        return qualitative_score, n_qual_items

    def compute_distress_notches(
        self,
        fin: Dict[str, float],
        altman_z: float,
    ) -> Tuple[int, Dict[str, float]]:
        total_notches = 0
        details: Dict[str, float] = {}

        ic = fin.get("interest_coverage")
        if ic is not None:
            for threshold, notches in DISTRESS_BANDS["interest_coverage"]:
                if ic < threshold:
                    total_notches += notches
                    details["interest_coverage"] = ic
                    break

        dscr = fin.get("dscr")
        if dscr is not None:
            for threshold, notches in DISTRESS_BANDS["dscr"]:
                if dscr < threshold:
                    total_notches += notches
                    details["dscr"] = dscr
                    break

        for threshold, notches in DISTRESS_BANDS["altman_z"]:
            if altman_z < threshold:
                total_notches += notches
                details["altman_z"] = altman_z
                break

        if total_notches < MAX_DISTRESS_NOTCHES:
            total_notches = MAX_DISTRESS_NOTCHES

        return total_notches, details

    def compute_final_rating(
        self,
        quant_inputs: QuantInputs,
        qual_inputs: QualInputs,
        sovereign_rating: Optional[str] = None,
        sovereign_outlook: Optional[str] = None,
        enable_hardstops: bool = False,
        enable_sovereign_cap: bool = False,
    ) -> RatingOutputs:
        # 1) Quantitative and qualitative scores
        quant_score, peer_score, bucket_avgs, altman_z, n_quant = self.compute_quantitative(quant_inputs)
        qual_score, n_qual = self.compute_qualitative(qual_inputs)

        # 2) Effective weights
        wq, wl = compute_effective_weights(n_quant, n_qual)
        logging.info(
            "%s-Weights: n_quant=%d n_qual=%d -> wq=%.3f wl=%.3f",
            self.cp_name, n_quant, n_qual, wq, wl
        )

        combined_score = wq * quant_score + wl * qual_score  # weighted average

        # 3) Base rating (model-only rating, no hardstops/cap)
        base_rating = safe_score_to_rating(combined_score)

        # 4) Hardstops / distress notches
        if enable_hardstops:
            distress_notches, hardstop_details = self.compute_distress_notches(
                quant_inputs.fin_t0,
                altman_z,
            )
        else:
            distress_notches = 0
            hardstop_details = {}

        hardstop_rating = move_notches(base_rating, distress_notches)
        hardstop_triggered = distress_notches < 0

        # 5) Sovereign cap application
        capped_rating = hardstop_rating
        if enable_sovereign_cap and sovereign_rating is not None:
            capped_rating = apply_sovereign_cap(hardstop_rating, sovereign_rating)

        final_rating = capped_rating  # currently no further adjustments

        # 6) Sovereign cap binding definition:
        #    binding if final rating equals sovereign rating and cap is enabled.
        sovereign_cap_binding = (
            enable_sovereign_cap
            and sovereign_rating is not None
            and final_rating == sovereign_rating
        )

        # 7) Outlook logic
        # 1) Band-based base outlook from score position within rating band
        base_outlook = derive_outlook_band_only(combined_score, base_rating)

        # 2) Sovereign-binding branch
        if (sovereign_cap_binding and sovereign_outlook in {"Positive", "Stable", "Negative"}):
            # Special aligned case: issuer rating == sovereign rating and same outlook
            # → keep model's band-based base_outlook
            if(
                hardstop_rating == capped_rating == sovereign_rating
                and base_outlook == sovereign_outlook
            ):
                outlook = base_outlook
            else:
                # Sovereign-aligned outlook when issuer is capped at or below sovereign
                if base_outlook == "Positive" and sovereign_outlook in {"Stable", "Negative"}:
                    # If model is more optimistic than the sovereign, sovereign dominates
                    outlook = sovereign_outlook
                elif base_outlook == "Negative" or sovereign_outlook == "Negative":
                    # if either side is Negative, keep it conservative
                    outlook = "Negative"
                else:
                    # Both sides non-Negative and not more optimistic than sovereign → Stable
                    outlook = "Stable"
            # 3) Non-binding / no-cap branch: distress-trend overlay
        else:
            # No binding: add distress trend logic on top of base_outlook
            # and only adjust if a distress hardstop actually bit (distress_notches < 0)
            outlook = derive_outlook_with_distress_trend(
                base_outlook,
                distress_notches,
                quant_inputs.fin_t0,
                quant_inputs.fin_t1,
            )

        # 4) Final guard: no Positive outlook at AAA
        if final_rating == "AAA" and outlook == "Positive":
            outlook = "Stable"

        # 8) Flags always present
        flags = {
            "enable_hardstops": enable_hardstops,
            "enable_sovereign_cap": enable_sovereign_cap and (sovereign_rating is not None),
            "hardstop_triggered": hardstop_triggered,
            "sovereign_cap_binding": sovereign_cap_binding,
        }

        # 9) Rating explanation (driven by flags and binding)
        parts: List[str] = []

        # Core model
        parts.append(
            f"Based on the quantitative and qualitative factors, the combined score is "
            f"{combined_score:.1f}, corresponding to a base rating of {base_rating}."
        )

        # Distress / hardstops
        if hardstop_triggered:
            parts.append(
                f" Distress factors {list(hardstop_details.keys())} triggered a total "
                f"of {abs(distress_notches)} notch(es) of downgrade, resulting in a "
                f"post-distress (hardstop) rating of {hardstop_rating}."
            )
        else:
            parts.append(
                f" No distress-related hardstops were applied, so the hardstop rating "
                f"remains equal to the base rating at {hardstop_rating}."
            )

        # Sovereign cap
        if enable_sovereign_cap and sovereign_rating is not None:
            if sovereign_cap_binding:
                if hardstop_rating != capped_rating:
                    # sovereign actively worsens the rating relative to hardstop
                    parts.append(
                        f" The sovereign cap is binding: given the sovereign rating of "
                        f"{sovereign_rating}, the rating is constrained from {hardstop_rating} "
                        f"to a capped rating of {capped_rating}."
                    )
                else:
                    # issuer is at sovereign level; cap is effectively binding at that level
                    parts.append(
                        f" The issuer's rating is aligned with the sovereign rating at "
                        f"{sovereign_rating}, so the sovereign cap is effectively binding."
                    )
            else:
                # cap present but not constraining
                parts.append(
                    f" A sovereign rating of {sovereign_rating} is considered, but it does not "
                    f"constrain the issuer rating, so the capped rating remains {capped_rating}."
                )
        else:
            # no cap applied
            parts.append(
                f" No sovereign cap is applied, so the capped rating is the same as the "
                f"post-distress rating at {capped_rating}."
            )

        # Final sentence
        parts.append(
            f" The final issuer rating is {final_rating} with an outlook of {outlook}."
        )

        rating_explanation = "".join(parts)

        logging.info(
            "%s-Final: base=%s hardstop=%s capped=%s final=%s outlook=%s "
            "distress_notches=%d",
            self.cp_name,
            base_rating,
            hardstop_rating,
            capped_rating,
            final_rating,
            outlook,
            distress_notches,
        )

        return RatingOutputs(
            issuer_name=self.cp_name,
            quantitative_score=quant_score,
            qualitative_score=qual_score,
            combined_score=combined_score,
            peer_score=peer_score,
            base_rating=base_rating,
            distress_notches=distress_notches,
            hardstop_rating=hardstop_rating,
            capped_rating=capped_rating,
            final_rating=final_rating,
            hardstop_triggered=hardstop_triggered,
            hardstop_details=hardstop_details,
            sovereign_rating=sovereign_rating,
            sovereign_outlook=sovereign_outlook,
            sovereign_cap_binding=sovereign_cap_binding,
            outlook=outlook,
            bucket_avgs=bucket_avgs,
            altman_z_t0=altman_z,
            flags=flags,
            rating_explanation=rating_explanation,
        )


In [39]:
# =========================
# Sample data and run
# =========================

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    fin_t0 = {
        "debt_ebitda": 3.2,
        "net_debt_ebitda": 2.8,
        "debt_equity": 1.5,
        "debt_capital": 0.55,
        "ffo_debt": 0.18,
        "fcf_debt": 0.12,
        "interest_coverage": 0.8,
        "fixed_charge_coverage": 1.4,
        "dscr": 0.95,
        "ebitda_margin": 0.18,
        "ebit_margin": 0.12,
        "roa": 0.055,
        "roe": 0.11,
        "capex_dep": 1.3,
        "current_ratio": 1.3,
        "rollover_coverage": 1.1,
    }

    fin_t1 = {
        "debt_ebitda": 3.6,
        "net_debt_ebitda": 3.1,
        "debt_equity": 1.6,
        "debt_capital": 0.57,
        "ffo_debt": 0.17,
        "fcf_debt": 0.10,
        "interest_coverage": 1.2,
        "fixed_charge_coverage": 1.6,
        "dscr": 1.05,
        "ebitda_margin": 0.165,
        "ebit_margin": 0.11,
        "roa": 0.052,
        "roe": 0.105,
        "capex_dep": 1.2,
        "current_ratio": 1.2,
        "rollover_coverage": 1.05,
    }

    fin_t2 = {
        "debt_ebitda": 3.9,
        "net_debt_ebitda": 3.4,
        "debt_equity": 1.7,
        "debt_capital": 0.60,
        "ffo_debt": 0.16,
        "fcf_debt": 0.09,
        "interest_coverage": 1.5,
        "fixed_charge_coverage": 1.8,
        "dscr": 1.10,
        "ebitda_margin": 0.155,
        "ebit_margin": 0.10,
        "roa": 0.050,
        "roe": 0.10,
        "capex_dep": 1.1,
        "current_ratio": 1.1,
        "rollover_coverage": 1.0,
    }

    components_t0 = {
        "working_capital": 120.0,
        "total_assets": 1000.0,
        "retained_earnings": 200.0,
        "ebit": 80.0,
        "market_value_equity": 600.0,
        "total_liabilities": 400.0,
        "sales": 900.0,
    }

    components_t1 = {
        "working_capital": 110.0,
        "total_assets": 950.0,
        "retained_earnings": 180.0,
        "ebit": 75.0,
        "market_value_equity": 580.0,
        "total_liabilities": 370.0,
        "sales": 880.0,
    }

    components_t2 = {
        "working_capital": 100.0,
        "total_assets": 900.0,
        "retained_earnings": 160.0,
        "ebit": 70.0,
        "market_value_equity": 550.0,
        "total_liabilities": 350.0,
        "sales": 860.0,
    }

    peers_t0 = {
        "debt_ebitda": [2.8, 3.0, 3.1],
        "net_debt_ebitda": [2.5, 2.7, 2.9],
        "debt_equity": [1.3, 1.4, 1.5],
        "debt_capital": [0.50, 0.52, 0.54],
        "ffo_debt": [0.20, 0.22, 0.24],
        "fcf_debt": [0.14, 0.16, 0.18],
        "interest_coverage": [2.0, 2.5, 3.0],
        "fixed_charge_coverage": [1.8, 2.0, 2.2],
        "dscr": [1.2, 1.3, 1.4],
        "ebitda_margin": [0.17, 0.18, 0.19],
        "ebit_margin": [0.115, 0.125, 0.13],
        "roa": [0.055, 0.06, 0.065],
        "roe": [0.11, 0.115, 0.12],
        "capex_dep": [1.1, 1.2, 1.3],
        "current_ratio": [1.2, 1.3, 1.4],
        "rollover_coverage": [1.1, 1.2, 1.3],
    }

    quant_inputs = QuantInputs(
        fin_t0=fin_t0,
        fin_t1=fin_t1,
        fin_t2=fin_t2,
        components_t0=components_t0,
        components_t1=components_t1,
        components_t2=components_t2,
        peers_t0=peers_t0,
    )

    qual_t0 = {
        "industry_risk": 3,
        "market_position": 5,
        "revenue_diversification": 5,
        "revenue_stability": 4,
        "business_model_resilience": 4,
        "management_quality": 4,
        "governance": 4,
        "financial_policy": 3,
        "sovereign_risk": 3,
        "legal_environment": 4,
        "transparency": 4,
        "liquidity_profile": 3,
        "wc_management_quality": 4,
        "refinancing_risk": 3,
    }
    qual_t1 = qual_t0.copy()

    qual_inputs = QualInputs(
        factors_t0=qual_t0,
        factors_t1=qual_t1,
    )

    sample_sovereign_rating = "A-"
    sample_sovereign_outlook = "Negative"

    model = RatingModel(cp_name="SampleCorp")
    out = model.compute_final_rating(
        quant_inputs,
        qual_inputs,
        sovereign_rating=sample_sovereign_rating,
        sovereign_outlook=sample_sovereign_outlook,
        enable_hardstops=False,
        enable_sovereign_cap=True,
    )

    summary = {
        "issuer_name": out.issuer_name,
        "quantitative_score": round(out.quantitative_score, 1),
        "qualitative_score": round(out.qualitative_score, 1),
        "combined_score": round(out.combined_score, 1),
        "peer_score": out.peer_score,
        "base_rating": out.base_rating,
        "distress_notches": out.distress_notches,
        "hardstop_rating": out.hardstop_rating,
        "final_rating": out.final_rating,
        "outlook": out.outlook,
        "hardstop_triggered": out.hardstop_triggered,
        "hardstop_details": out.hardstop_details,
        "sovereign_rating": out.sovereign_rating,
        "sovereign_outlook": out.sovereign_outlook,
        "bucket_avgs": out.bucket_avgs,
        "altman_z_t0": out.altman_z_t0,
        "flags": out.flags,
        "rating_explanation": out.rating_explanation,
    }

    for k, v in summary.items():
        print(f"{k}: {v}")


INFO:root:SampleCorp-AltmanZ: computed z=2.488 from components
INFO:root:SampleCorp-Quant: debt_ebitda value=3.200 score=50.0 family=leverage
INFO:root:SampleCorp-Quant: net_debt_ebitda value=2.800 score=75.0 family=leverage
INFO:root:SampleCorp-Quant: debt_equity value=1.500 score=50.0 family=leverage
INFO:root:SampleCorp-Quant: debt_capital value=0.550 score=25.0 family=leverage
INFO:root:SampleCorp-Quant: ffo_debt value=0.180 score=50.0 family=leverage_rev
INFO:root:SampleCorp-Quant: fcf_debt value=0.120 score=75.0 family=leverage_rev
INFO:root:SampleCorp-Quant: interest_coverage value=0.800 score=0.0 family=coverage
INFO:root:SampleCorp-Quant: fixed_charge_coverage value=1.400 score=0.0 family=coverage
INFO:root:SampleCorp-Quant: dscr value=0.950 score=0.0 family=coverage
INFO:root:SampleCorp-Quant: ebitda_margin value=0.180 score=75.0 family=profit
INFO:root:SampleCorp-Quant: ebit_margin value=0.120 score=75.0 family=profit
INFO:root:SampleCorp-Quant: roa value=0.055 score=50.0 fa

issuer_name: SampleCorp
quantitative_score: 48.6
qualitative_score: 69.6
combined_score: 57.8
peer_score: 50.0
base_rating: BBB
distress_notches: 0
hardstop_rating: BBB
final_rating: BBB
outlook: Stable
hardstop_triggered: False
hardstop_details: {}
sovereign_rating: A-
sovereign_outlook: Negative
bucket_avgs: {'leverage': 50.0, 'leverage_rev': 62.5, 'coverage': 0.0, 'profit': 62.5, 'other': 62.5, 'altman': 50.0}
altman_z_t0: 2.488
flags: {'enable_hardstops': False, 'enable_sovereign_cap': True, 'hardstop_triggered': False, 'sovereign_cap_binding': False}
rating_explanation: Based on the quantitative and qualitative factors, the combined score is 57.8, corresponding to a base rating of BBB. No distress-related hardstops were applied, so the hardstop rating remains equal to the base rating at BBB. A sovereign rating of A- is considered, but it does not constrain the issuer rating, so the capped rating remains BBB. The final issuer rating is BBB with an outlook of Stable.
